In [1]:
# Install necessary libraries if not already installed
# Note: Uncomment the next line if you encounter issues with imports
# !pip install google-cloud-vision google-auth google-auth-oauthlib google-api-python-client torch torchvision

import requests
import base64
from google.cloud import vision
from google.oauth2 import service_account
from googleapiclient.discovery import build
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import io

# TODO -> put the path to your API key and service account file
api_key_file_path = '/Users/angelokaram/Desktop/Sac State Folders/Sac State Spring 2024/CSC 190/Senior-Project/private_keys.txt'
SERVICE_ACCOUNT_FILE = '/Users/angelokaram/Desktop/Sac State Folders/Sac State Spring 2024/CSC 190/Senior-Project/test-ai-sheet-1cbe18b0629d.json'

# Read the API key from the file
def get_api_key_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            if len(lines) >= 2:
                return lines[0].strip()  # API key on the first line
            else:
                raise ValueError("API key not found in the file.")
    except Exception as e:
        print(f"Failed to read the API key from file: {e}")
        return None

# Authentication and setup
API_KEY = get_api_key_from_file(api_key_file_path)
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=credentials)

# Spreadsheet details
SPREADSHEET_ID = '157Dpz8bSe0NMoUAcElBKG6InbRuiboWqht11-tdlnNA'
READ_RANGE_NAME = 'Sheet1!A2'  # Read Column A, Row 2
WRITE_RANGE_NAME = 'Sheet1!D2'  # Write to Column D, Row 2

# Preprocess the image for CNN (grayscale, resized, normalized)
def preprocess_image(image_url):
    response = requests.get(image_url)
    img = Image.open(io.BytesIO(response.content)).convert('L')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize to 256x256
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values to range [-1, 1]
    ])
    return transform(img).unsqueeze(0)  # Add batch dimension

# CNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # Conv layer 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Conv layer 2
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling (2x2)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, 10)  # Output layer (10 classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Apply conv1 + ReLU + maxpool
        x = self.pool(torch.relu(self.conv2(x)))  # Apply conv2 + ReLU + maxpool
        x = x.view(-1, 64 * 64 * 64)  # Flatten feature maps into a vector
        x = torch.relu(self.fc1(x))  # Fully connected layer 1 with ReLU
        x = torch.sigmoid(self.fc2(x))  # Output layer with sigmoid (binary classification)
        return x

# Instantiate the CNN model
cnn_model = SimpleCNN()

# Loss function and optimizer for CNN
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=0.001, momentum=0.9)

# perform image recognition using CNN
def cnn_image_recognition(image_url):
    # Preprocess the image
    image_tensor = preprocess_image(image_url)
    
    # Forward pass through the CNN model
    cnn_model.eval()  # Set the model to evaluation mode (inference mode)
    with torch.no_grad():
        output = cnn_model(image_tensor)  # Get the CNN's output
        predicted_class = torch.argmax(output, dim=1)  # Get the predicted class (index of max logit)

    # Return the predicted class
    return predicted_class.item()

# Detect text using Google Vision API
def detect_text_via_api(image_url):
    url = f"https://vision.googleapis.com/v1/images:annotate?key={API_KEY}"
    headers = {'Content-Type': 'application/json'}
    image_content = encode_image(image_url)
    
    body = {
        "requests": [
            {
                "image": {
                    "content": image_content
                },
                "features": [
                    {"type": "DOCUMENT_TEXT_DETECTION"}  # Text detection (good for handwriting)
                ]
            }
        ]
    }

    # API request and response handling
    response = requests.post(url, headers=headers, json=body)
    result = response.json()
    print(result)  # Debugging: prints the API response

    if 'responses' in result and len(result['responses']) > 0 and 'textAnnotations' in result['responses'][0]:
        annotations = result['responses'][0]['textAnnotations']
        detected_text = annotations[0]['description']  # Get the first detected text block
        print(f"Detected text: {detected_text}")
        return detected_text
    else:
        return 'No text detected'

# Image encoding for Google Vision API
def encode_image(image_url):
    """Fetches an image from the URL and encodes it in base64."""
    try:
        response = requests.get(image_url)
        response.raise_for_status()  # raise exception for HTTP errors
        return base64.b64encode(response.content).decode()
    except requests.RequestException as e:
        print(f"Failed to fetch image from URL: {e}")
        return None

# read the image URL from Google Sheets and perform text and CNN recognition
def read_and_update_sheet():
    # Read the image URL from the sheet
    result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=READ_RANGE_NAME).execute()
    image_url = result.get('values', [[None]])[0][0]
    if image_url:
        # Use Google Vision API for text detection
        detected_text = detect_text_via_api(image_url)
        
        # Use CNN for image classification
        predicted_class = cnn_image_recognition(image_url)
        print(f"Predicted class from CNN: {predicted_class}")
        
        # Write both detected text and CNN class back to the sheet
        values = [[detected_text, predicted_class]]
        body = {'values': values}
        update_result = service.spreadsheets().values().update(
            spreadsheetId=SPREADSHEET_ID, range=WRITE_RANGE_NAME,
            valueInputOption='USER_ENTERED', body=body).execute()
        print(f'Cells updated: {update_result.get("updatedCells")}')
    else:
        print("No URL found in the specified cell.")

# runmain function
read_and_update_sheet()

def test_read_and_update_sheet():
    read_and_update_sheet()
    # checks the Google Sheet to ensure the results were written correctly
    result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=WRITE_RANGE_NAME).execute()
    updated_values = result.get('values', [])
    assert updated_values is not None, "No values found in the updated Google Sheet."
    assert len(updated_values) > 0, "No updated values found in the sheet."



Failed to fetch image from URL: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /src/assests/dogcat.png (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1294535c0>: Failed to establish a new connection: [Errno 61] Connection refused'))
{'error': {'code': 400, 'message': 'Request must specify image and features.', 'status': 'INVALID_ARGUMENT'}}


ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /src/assests/dogcat.png (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x129451670>: Failed to establish a new connection: [Errno 61] Connection refused'))